In [1]:
import brightway2 as bw
import os               # to use "operating system dependent functionality"
import numpy as np      # "the fundamental package for scientific computing with Python"
import pandas as pd     # "high-performance, easy-to-use data structures and data analysis tools" for Python
import time

In [2]:
#bw.projects.set_current("SpLCA_farm2020")
bw.projects.set_current("farm2020")

In [3]:
bw.projects.dir
#bw.projects.logs_dir
#bw.projects.current

'C:\\Users\\ruqun\\AppData\\Local\\pylca\\Brightway3\\farm2020.e3bfae4344258b666791e44d43a2724d'

In [4]:
bw.Database('biosphere3')

Brightway2 SQLiteBackend: biosphere3

In [5]:
bw.Database('ecoinvent 3.6 cutoff')

Brightway2 SQLiteBackend: ecoinvent 3.6 cutoff

In [6]:
my_bio = bw.Database('biosphere3')
#len(my_bio)  #4321

In [7]:
eidb = bw.Database('ecoinvent 3.6 cutoff')
#len(eidb)   #original 18121 to 18144

In [8]:
bw.databases        #3.3 was duplicate of 3.6

Databases dictionary with 3 object(s):
	biosphere3
	ecoinvent 3.3 cutoff
	ecoinvent 3.6 cutoff

# create empty activity (to be used in foreground) and search ecoinvent  

In [9]:
#define farm # here in x: 
x = 60
farmno = str(x)

#delete all activities that were accidentally created

In [77]:
for activity in [act for act in eidb if act['name']== "farm" + farmno +"_upstream_allinput" ]:
    activity.delete()

In [78]:
for activity in [act for act in eidb if act['name']== "farm" + farmno +"_onfarmprocess" ]:
    activity.delete()

In [79]:
for activity in [act for act in eidb if act['name']== "farm" + farmno + '_onsite_dieselburn']:
    activity.delete()

In [80]:
for activity in [act for act in eidb if act['name']== "farm" + farmno + '_onsite_coalburn']:
    activity.delete()

In [81]:
for activity in [act for act in eidb if act['name']== "farm" + farmno + '_onsite_animalemi']:
    activity.delete()

In [82]:
globals()['upstream_farm%s' % x] = eidb.new_activity(code = "farmtest"+farmno+"_upstream", name = "farm" + farmno +"_upstream_allinput", unit = "unit")
globals()['upstream_farm%s' % x].save()

In [83]:
globals()['onsite_farm%s' % x] = eidb.new_activity(code = "farmtest"+farmno+"_processing", name = "farm" + farmno +"_onfarmprocess", unit = "unit")
globals()['onsite_farm%s' % x].save()

In [84]:
globals()['onsite_diesel_farm%s' % x] = eidb.new_activity(code = "farmtest"+farmno+"_processing_diesel", name = "farm" + farmno +"_onsite_dieselburn", unit = "unit")
globals()['onsite_diesel_farm%s' % x].save()

In [85]:
globals()['onsite_coal_farm%s' % x] = eidb.new_activity(code = "farmtest"+farmno+"_processing_coal", name = "farm" + farmno +"_onsite_coalburn", unit = "unit")
globals()['onsite_coal_farm%s' % x].save()

In [86]:
globals()['onsite_animalemi_farm%s' % x] = eidb.new_activity(code = "farmtest"+farmno+"_processing_animal", name = "farm" + farmno +"_onsite_animalemi", unit = "unit")
globals()['onsite_animalemi_farm%s' % x].save()

In [20]:
# 1 MJ (0.0222 kg) diesel burned for running a tractor with a trailer.
diesel_burn = [act for act in eidb if act['name']=='diesel, burned in agricultural machinery'][0]

In [21]:
farm_diesel_burnEmi = diesel_burn.copy()

In [22]:
farm_diesel_burnEmi.technosphere().delete()   #delete tech input from original ecoinvent and leave only burning emissions
#for exc in farm_diesel_burnEmi.exchanges(): print(exc)

In [23]:
#farm_coal_burnEmi = eidb.new_activity(code = 'farmtest_coalburn', name = "farm_onsite_coalburn", unit = "megajoule")
#farm_diesel_burnEmi.save()

In [24]:
coal_burn = [act for act in eidb if act['name']=='heat production, lignite briquette, at stove 5-15kW' and 'RoW' in act['location']][0]

In [25]:
farm_coal_burnEmi = coal_burn.copy() #heating value (19.5 MJ/kg), unit: megajoule

In [26]:
farm_coal_burnEmi.technosphere().delete()
#for exc in farm_coal_burnEmi.technosphere(): print(exc)

### upstream production (not use copy())

In [27]:
#upstream production
#kilowatt hour
farmx_elec = [act for act in eidb if act['name']=='electricity production, hard coal' and 'CN-NM' in act['location']][0]
#megajoule
farmx_diesel = [act for act in eidb if act['name']=='market for diesel, burned in agricultural machinery' and 'GLO' in act['location']][0]
#megajoule
farmx_coal = [act for act in eidb if act['name']=='market for lignite briquettes' and 'RoW' in act['location']][0]

#feed options, pelletfeed and concentrated feed using "market for protein feed" 
#（玉米粉/玉米糠/玉米面 maizechop）（玉米皮/玉米饼/玉米粒/玉米料/喷浆玉米 maizegrain）   (青贮玉米 maize silage)
farmx_pelletfeed = [act for act in eidb if 'market for protein feed, 100% crude' in act['name']][0]
farmx_soybeanmeal = [act for act in eidb if act['name']=='soybean meal to generic market for protein feed' and act['unit'] == 'kilogram'][0]
farmx_cottonseedmeal = [act for act in eidb if act['name']=='cottonseed meal to generic market for protein feed' and act['unit'] == 'kilogram'][0]
farmx_vinasse = [act for act in eidb if 'Dried Grains with Solubles to generic market for protein feed' in act['name'] and act['unit'] == 'kilogram'][0]
farmx_rapemeal = [act for act in eidb if act['name']=='rape meal to generic market for protein feed' and act['unit'] == 'kilogram'][0]
farmx_wheatbran = [act for act in eidb if act['name']=='wheat bran to generic market for energy feed' and act['unit'] == 'kilogram' and 'RoW' in act['location']][0]
farmx_maizechop = [act for act in eidb if act['name']=='maize chop to generic market for energy feed' and act['unit'] == 'kilogram' and 'RoW' in act['location']][0]
farmx_hay = [act for act in eidb if act['name'] == 'market for hay'][0]
farmx_straw = [act for act in eidb if 'market for straw, stand-alone production' in act['name'] and 'RoW' in act['location']][0]
farmx_maizesilage = [act for act in eidb if 'market for maize silage' in act['name'] and 'RoW' in act['location']][0]
farmx_grasssilage = [act for act in eidb if 'grass silage production' in act['name'] and 'RoW' in act['location']][0]
farmx_maizegrain = [act for act in eidb if act['name']=='market for maize grain' and 'RoW' in act['location']][0]
farmx_milk = [act for act in eidb if act['name']=='skimmed milk, from cow milk to generic market for protein feed' and act['unit'] == 'kilogram'][0]
#farmx_wheatgrain = [act for act in eidb if act['name']=='wheat grain, feed production' and 'RoW' in act['location']][0]
#farmx_barleygrain = [act for act in eidb if act['name']=='barley grain, feed production' and 'RoW' in act['location']][0]
#farmx_meatbonemeal = [act for act in eidb if act['name']=='meat and bone meal to generic market for protein feed' and act['unit'] == 'kilogram' and 'RoW' in act['location']][0] 


### natural resources input

In [28]:
#bbb = [act for act in eidb if act['name']== 'beef cattle production on pasture' and 'RoW' in act['location']][0]
#for exc in bbb.biosphere(): print(exc)
#Exchange: 0.00474868029053916 cubic meter 'Water, unspecified natural origin' (cubic meter, None, ('natural resource', 'in ground')) to 'beef cattle production on pasture' (kilogram, RoW, None)>
#Exchange: 106.602633613578 square meter-year 'Occupation, pasture, man made' (square meter-year, None, ('natural resource', 'land')) to 'beef cattle production on pasture' (kilogram, RoW, None)>
#Exchange: 37.6747149756226 square meter-year 'Occupation, grassland, natural, for livestock grazing' (square meter-year, None, ('natural resource', 'land')) to 'beef cattle production on pasture' (kilogram, RoW, None)>

In [87]:
#water, cubic meter
farmx_input_water_unspecified = [act for act in my_bio if act['name']=='Water, unspecified natural origin' and act['type'] =='natural resource'
  and 'natural resource' in str(act['categories']) and 'in water' in str(act['categories'])][0]
#farmx_input_water_river = [act for act in my_bio if act['name']=='Water, river' and act['type'] =='natural resource'
#  and 'natural resource' in str(act['categories']) and 'in water' in str(act['categories'])][0]
#farmx_input_water_lake = [act for act in my_bio if act['name']=='Water, lake' and act['type'] =='natural resource'
#  and 'natural resource' in str(act['categories']) and 'in water' in str(act['categories'])][0]

In [88]:
#pasture, square meter-year
farmx_input_pasture = [act for act in my_bio if act['name'] == 'Occupation, grassland, natural, for livestock grazing'][0]

### direct onsite emissions from DairyGEM model

In [89]:
#direct onsite emissions
#my_bio.search('ammonia')
#bw.Database('biosphere3').search('carbon dioxide', filter={'categories':'urban', 'name':'fossil'})
#bw.Database('biosphere3').search('ammonia', filter={'categories':'non-urban'})
farmx_emi_ammonia = [act for act in my_bio if act['name']=='Ammonia' and 'air' and 'non-urban air or from high stacks' in str(act['categories'])][0]
farmx_emi_hydrosulfide = [act for act in my_bio if act['name']=='Hydrogen sulfide' and 'air' in str(act['categories']) and 'non-urban air or from high stacks' in str(act['categories'])][0]
farmx_emi_methane = [act for act in my_bio if act['name']=='Methane'][0]
farmx_emi_VOC = [act for act in my_bio if 'NMVOC, non-methane volatile organic compounds, unspecified origin' in act['name'] and 'non-urban air or from high stacks' in str(act['categories'])][0]
farmx_emi_nitroxides = [act for act in my_bio if 'Nitrogen oxides' in act['name'] and 'non-urban air or from high stacks' in str(act['categories'])][0]
#farmx_emi_bioCO2 = [act for act in my_bio if 'Carbon dioxide, non-fossil' in act['name'] and 'non-urban air or from high stacks' in str(act['categories'])][0]
#farmx_emi_soil = [act for act in my_bio if act['name']=='Nitrate' and 'soil' in str(act['categories'])][0]
#farmx_emi_nitrate_water = [act for act in my_bio if act['name']=='Nitrate' and 'water' in str(act['categories'])][0]

# Creating foreground system for farmx

In [90]:
#define FU for each farm here, do not touch following codes
farmx_income = 47.9*10000      #RMB yuan
farmx_liveweight = 28.14*1000     #kg

In [91]:
#feed input, change amount (kg) for each farm here
pelletfeed_amt = 0
soybeanmeal_amt =  0
cottonseedmeal_amt = 0
vinasse_amt = 0
rapemeal_amt = 1600
wheatbran_amt = 0
maizechop_amt =  0
meatbonemeal_amt = 0
hay_amt = 25*1000
straw_amt = 0
maizesilage_amt = 0
grasssilage_amt = 0 
wheatgrain_amt = 0 
barleygrain_amt = 0
maizegrain_amt = 0
milk_amt = 0

#resource input
water_amt = 359598                          #m3
land_amt = 166 * 10000                     #ha to m2

#energy input 
elec_amt = 3600           
coal_amt_kg = 15000         #convert to MJ
coal_amt_MJ = coal_amt_kg * 19.5   #19.5 MJ/kg for coal
diesel_amt_liter = 2400    #convert to MJ
diesel_amt_MJ = diesel_amt_liter * 0.832 * 45.045  #45.045 MJ/kg  | 0.832kg/L for diesel

#farm emission output (annual kg)
ammonia_amt = 473
hydrosulfide_amt = 3
methane_amt = 4151
nitroxides_amt = 4
VOC_amt = 48

In [92]:
#technosphere input all written to farmx_upstream
globals()['upstream_farm%s' % x].new_exchange(input=farmx_elec.key,amount=elec_amt,unit="kilowatt hour",type='technosphere').save()
globals()['upstream_farm%s' % x].new_exchange(input=farmx_diesel.key,amount=diesel_amt_MJ,unit="megajoule",type='technosphere').save()
globals()['upstream_farm%s' % x].new_exchange(input=farmx_coal.key,amount=coal_amt_MJ,unit="megajoule",type='technosphere').save()
globals()['upstream_farm%s' % x].new_exchange(input=farmx_pelletfeed.key,amount=pelletfeed_amt,unit="kilogram",type='technosphere').save()
globals()['upstream_farm%s' % x].new_exchange(input=farmx_soybeanmeal.key,amount=soybeanmeal_amt,unit="kilogram",type='technosphere').save()
globals()['upstream_farm%s' % x].new_exchange(input=farmx_cottonseedmeal.key,amount=cottonseedmeal_amt,unit="kilogram",type='technosphere').save()
globals()['upstream_farm%s' % x].new_exchange(input=farmx_vinasse.key,amount=vinasse_amt,unit="kilogram",type='technosphere').save()
globals()['upstream_farm%s' % x].new_exchange(input=farmx_rapemeal.key,amount=rapemeal_amt,unit="kilogram",type='technosphere').save()
globals()['upstream_farm%s' % x].new_exchange(input=farmx_wheatbran.key,amount=wheatbran_amt,unit="kilogram",type='technosphere').save()
globals()['upstream_farm%s' % x].new_exchange(input=farmx_maizechop.key,amount=maizechop_amt,unit="kilogram",type='technosphere').save()
globals()['upstream_farm%s' % x].new_exchange(input=farmx_hay.key,amount=hay_amt,unit="kilogram",type='technosphere').save()
globals()['upstream_farm%s' % x].new_exchange(input=farmx_straw.key,amount=straw_amt,unit="kilogram",type='technosphere').save()
globals()['upstream_farm%s' % x].new_exchange(input=farmx_maizesilage.key,amount=maizesilage_amt,unit="kilogram",type='technosphere').save()
globals()['upstream_farm%s' % x].new_exchange(input=farmx_grasssilage.key,amount=grasssilage_amt,unit="kilogram",type='technosphere').save()
globals()['upstream_farm%s' % x].new_exchange(input=farmx_maizegrain.key,amount=maizegrain_amt,unit="kilogram",type='technosphere').save()
globals()['upstream_farm%s' % x].new_exchange(input=farmx_milk.key,amount=milk_amt,unit="kilogram",type='technosphere').save()
#globals()['upstream_farm%s' % x].save()

on-site processes

In [93]:
#direct emissions from diesel and coal burning, each for (1MJ) emission
globals()['onsite_diesel_farm%s' % x].new_exchange(input=farm_diesel_burnEmi.key,amount=diesel_amt_MJ,unit="megajoule",type='technosphere').save()
#farm1_processing.save()

In [94]:
globals()['onsite_coal_farm%s' % x].new_exchange(input=farm_coal_burnEmi.key,amount=coal_amt_MJ,unit="megajoule",type='technosphere').save()

In [95]:
#direct animal emissions 
globals()['onsite_animalemi_farm%s' % x].new_exchange(input=farmx_emi_ammonia.key,amount=ammonia_amt,unit="kilogram",type='biosphere').save()
globals()['onsite_animalemi_farm%s' % x].new_exchange(input=farmx_emi_hydrosulfide.key,amount=hydrosulfide_amt,unit="kilogram",type='biosphere').save()
globals()['onsite_animalemi_farm%s' % x].new_exchange(input=farmx_emi_methane.key,amount=methane_amt,unit="kilogram",type='biosphere').save()
globals()['onsite_animalemi_farm%s' % x].new_exchange(input=farmx_emi_VOC.key,amount=VOC_amt,unit="kilogram",type='biosphere').save()
globals()['onsite_animalemi_farm%s' % x].new_exchange(input=farmx_emi_nitroxides.key,amount=nitroxides_amt,unit="kilogram",type='biosphere').save()

In [96]:
#onsite resource input
globals()['onsite_farm%s' % x].new_exchange(input=farmx_input_pasture.key,amount=land_amt,unit="square meter-year",type='biosphere').save()
globals()['onsite_farm%s' % x].new_exchange(input=farmx_input_water_unspecified.key,amount=water_amt,unit="cubic meter",type='biosphere').save()

In [97]:
#for exc in globals()['upstream_farm%s' % x].exchanges():          print(exc)

In [98]:
for exc in globals()['onsite_diesel_farm%s' % x].exchanges():  
        print(exc)

Exchange: 89945.856 megajoule 'diesel, burned in agricultural machinery' (megajoule, GLO, None) to 'farm60_onsite_dieselburn' (unit, GLO, None)>


In [99]:
for exc in globals()['onsite_coal_farm%s' % x].exchanges():  
        print(exc)

Exchange: 292500.0 megajoule 'heat production, lignite briquette, at stove 5-15kW' (megajoule, RoW, None) to 'farm60_onsite_coalburn' (unit, GLO, None)>


In [100]:
for exc in globals()['onsite_animalemi_farm%s' % x].exchanges():  
        print(exc)

Exchange: 473 kilogram 'Ammonia' (kilogram, None, ('air', 'non-urban air or from high stacks')) to 'farm60_onsite_animalemi' (unit, GLO, None)>
Exchange: 3 kilogram 'Hydrogen sulfide' (kilogram, None, ('air', 'non-urban air or from high stacks')) to 'farm60_onsite_animalemi' (unit, GLO, None)>
Exchange: 4151 kilogram 'Methane' (kilogram, None, ('air', 'urban air close to ground')) to 'farm60_onsite_animalemi' (unit, GLO, None)>
Exchange: 48 kilogram 'NMVOC, non-methane volatile organic compounds, unspecified origin' (kilogram, None, ('air', 'non-urban air or from high stacks')) to 'farm60_onsite_animalemi' (unit, GLO, None)>
Exchange: 4 kilogram 'Nitrogen oxides' (kilogram, None, ('air', 'non-urban air or from high stacks')) to 'farm60_onsite_animalemi' (unit, GLO, None)>


In [101]:
for exc in globals()['onsite_farm%s' % x].exchanges():  
        print(exc)

Exchange: 1660000 square meter-year 'Occupation, grassland, natural, for livestock grazing' (square meter-year, None, ('natural resource', 'land')) to 'farm60_onfarmprocess' (unit, GLO, None)>
Exchange: 359598 cubic meter 'Water, unspecified natural origin' (cubic meter, None, ('natural resource', 'in water')) to 'farm60_onfarmprocess' (unit, GLO, None)>


# LCA farmx_upstream, onsite_coal, diesel,animalemi, resource

In [102]:
#FU income
GWP = [m for m in bw.methods if 'IPCC 2013' in str(m) and  'climate change' in str(m) and 'GWP 100a' in str(m)][0]
FUincome_upstream = {globals()['upstream_farm%s' % x]: (1/farmx_income)}
FUincome_onsite = {globals()['onsite_farm%s' % x]: (1/farmx_income)}
FUincome_onsite_diesel = {globals()['onsite_diesel_farm%s' % x]: (1/farmx_income)}
FUincome_onsite_coal = {globals()['onsite_coal_farm%s' % x]: (1/farmx_income)}
FUincome_onsite_animalemi = {globals()['onsite_animalemi_farm%s' % x]: (1/farmx_income)}

In [103]:
Fossil = [m for m in bw.methods if 'cumulative energy demand' in str(m) and 'fossil' in str(m) and 'non-renewable energy resources' in str(m)][0]

In [104]:
#FU protein
#FUprotein_upstream = {globals()['upstream_farm%s' % x]:(1/farmx_income)}
#FUprotein_onsite = {globals()['onsite_farm%s' % x]:(1/farmx_income)}

In [105]:
globals()['upstreamLCA_farm%s' % x] = bw.LCA(FUincome_upstream, GWP)

In [106]:
globals()['onsiteLCA_farm%s' % x] = bw.LCA(FUincome_onsite, GWP)   #onsite resources input only 

In [107]:
globals()['onsite_diesel_farm%s' % x] = bw.LCA(FUincome_onsite_diesel, GWP)

In [108]:
globals()['onsite_coal_farm%s' % x] = bw.LCA(FUincome_onsite_coal, GWP)

In [109]:
globals()['onsite_animalemi_farm%s' % x] = bw.LCA(FUincome_onsite_animalemi, GWP)

#lci() for all activities: 

In [110]:
globals()['upstreamLCA_farm%s' % x].lci()  #upstreamLCA_farmx.lci()

In [111]:
globals()['onsiteLCA_farm%s' % x].lci()  #onsiteLCA_farmx.lci()

In [112]:
globals()['onsite_diesel_farm%s' % x].lci() 

In [113]:
globals()['onsite_coal_farm%s' % x].lci() 

In [114]:
globals()['onsite_animalemi_farm%s' % x].lci() 

#record lci

In [115]:
farmxLCA_upstream_rev_activity_dict = {value:key for key, value in globals()['upstreamLCA_farm%s' % x].activity_dict.items()}
farmxLCA_upstream_rev_activity_dict
farmxLCA_upstream_rev_act_dict, farmxLCA_upstream_rev_product_dict, farmxLCA_upstream_rev_bio_dict = globals()['upstreamLCA_farm%s' % x].reverse_dict()

In [116]:
lci_nrow = len(farmxLCA_upstream_rev_bio_dict)
lci_nrow

2089

In [117]:
farmxLCA_onsite_rev_activity_dict = {value:key for key, value in globals()['onsiteLCA_farm%s' % x].activity_dict.items()}
farmxLCA_onsite_rev_activity_dict
farmxLCA_onsite_rev_act_dict, farmxLCA_onsite_rev_product_dict, farmxLCA_onsite_rev_bio_dict = globals()['onsiteLCA_farm%s' % x].reverse_dict()

In [118]:
farmxLCA_onsite_diesel_rev_activity_dict = {value:key for key, value in globals()['onsite_diesel_farm%s' % x].activity_dict.items()}
farmxLCA_onsite_diesel_rev_activity_dict
farmxLCA_onsite_diesel_rev_act_dict, farmxLCA_onsite_diesel_rev_product_dict, farmxLCA_onsite_diesel_rev_bio_dict = globals()['onsite_diesel_farm%s' % x].reverse_dict()

In [119]:
farmxLCA_onsite_coal_rev_activity_dict = {value:key for key, value in globals()['onsite_coal_farm%s' % x].activity_dict.items()}
farmxLCA_onsite_coal_rev_activity_dict
farmxLCA_onsite_coal_rev_act_dict, farmxLCA_onsite_coal_rev_product_dict, farmxLCA_onsite_coal_rev_bio_dict = globals()['onsite_coal_farm%s' % x].reverse_dict()

In [120]:
farmxLCA_onsite_animalemi_rev_activity_dict = {value:key for key, value in globals()['onsite_animalemi_farm%s' % x].activity_dict.items()}
farmxLCA_onsite_animalemi_rev_activity_dict
farmxLCA_onsite_animalemi_rev_act_dict, farmxLCA_onsite_animalemi_rev_product_dict, farmxLCA_onsite_animalemi_rev_bio_dict = globals()['onsite_animalemi_farm%s' % x].reverse_dict()

In [121]:
globals()['upstreamLCA_farm%s' % x].technosphere_matrix

<18249x18249 sparse matrix of type '<class 'numpy.float64'>'
	with 208525 stored elements in Compressed Sparse Row format>

In [122]:
globals()['onsiteLCA_farm%s' % x].technosphere_matrix

<18249x18249 sparse matrix of type '<class 'numpy.float64'>'
	with 208525 stored elements in Compressed Sparse Row format>

GWP calculation

In [123]:
globals()['onsiteLCA_farm%s' % x].lcia()
globals()['onsiteLCA_farm%s' % x].score

0.0

In [124]:
globals()['onsite_diesel_farm%s' % x].lcia()
globals()['onsite_diesel_farm%s' % x].score

0.013264322818438526

In [125]:
globals()['onsite_coal_farm%s' % x].lcia()
globals()['onsite_coal_farm%s' % x].score

0.1018838626512868

In [126]:
globals()['onsite_animalemi_farm%s' % x].lcia()
globals()['onsite_animalemi_farm%s' % x].score

0.0

In [127]:
globals()['upstreamLCA_farm%s' % x].lcia()
globals()['upstreamLCA_farm%s' % x].score

0.0780150069920647

Fossil energy

In [128]:
globals()['upstreamLCA_farm%s' % x] = bw.LCA(FUincome_upstream, Fossil)
globals()['upstreamLCA_farm%s' % x].lci()
globals()['upstreamLCA_farm%s' % x].lcia()
globals()['upstreamLCA_farm%s' % x].score

1.4697596399814385

# final LCI output

In [129]:
globals()['upstreamLCA_farm%s' % x].inventory

<2089x18249 sparse matrix of type '<class 'numpy.float64'>'
	with 295665 stored elements in Compressed Sparse Row format>

In [130]:
globals()['onsiteLCA_farm%s' % x].inventory

<2089x18249 sparse matrix of type '<class 'numpy.float64'>'
	with 2 stored elements in Compressed Sparse Row format>

In [131]:
globals()['onsite_diesel_farm%s' % x].inventory

<2089x18249 sparse matrix of type '<class 'numpy.float64'>'
	with 22 stored elements in Compressed Sparse Row format>

In [132]:
globals()['onsite_coal_farm%s' % x].inventory

<2089x18249 sparse matrix of type '<class 'numpy.float64'>'
	with 72 stored elements in Compressed Sparse Row format>

In [133]:
globals()['onsite_animalemi_farm%s' % x].inventory

<2089x18249 sparse matrix of type '<class 'numpy.float64'>'
	with 5 stored elements in Compressed Sparse Row format>

#store LCI values

In [134]:
LCI_cradle_to_gate_upstream = globals()['upstreamLCA_farm%s' % x].inventory.sum(axis=1)   #df.sum(axis=1) -> a row sum; 
LCI_cradle_to_gate_upstream.shape

(2089, 1)

In [135]:
LCI_cradle_to_gate_onsite = globals()['onsiteLCA_farm%s' % x].inventory.sum(axis=1)   #df.sum(axis=1) -> a row sum; 
LCI_cradle_to_gate_onsite.shape

(2089, 1)

In [136]:
LCI_cradle_to_gate_onsite_diesel = globals()['onsite_diesel_farm%s' % x].inventory.sum(axis=1)   
LCI_cradle_to_gate_onsite_diesel.shape

(2089, 1)

In [137]:
LCI_cradle_to_gate_onsite_coal = globals()['onsite_coal_farm%s' % x].inventory.sum(axis=1)   
LCI_cradle_to_gate_onsite_coal.shape

(2089, 1)

In [138]:
LCI_cradle_to_gate_onsite_animalemi = globals()['onsite_animalemi_farm%s' % x].inventory.sum(axis=1)   
LCI_cradle_to_gate_onsite_animalemi.shape

(2089, 1)

#store LCI names and categories

In [139]:
allBioNames_upstream = [bw.get_activity(farmxLCA_upstream_rev_bio_dict[r])['name'] for r in range(lci_nrow)]
allBioCompartment_upstream = [bw.get_activity(farmxLCA_upstream_rev_bio_dict[r])['categories'] for r in range(lci_nrow)]

In [140]:
allBioNames_onsite = [bw.get_activity(farmxLCA_onsite_rev_bio_dict[r])['name'] for r in range(lci_nrow)]
allBioCompartment_onsite = [bw.get_activity(farmxLCA_onsite_rev_bio_dict[r])['categories'] for r in range(lci_nrow)]

In [141]:
allBioNames_onsite_diesel = [bw.get_activity(farmxLCA_onsite_diesel_rev_bio_dict[r])['name'] for r in range(lci_nrow)]
allBioCompartment_onsite_diesel = [bw.get_activity(farmxLCA_onsite_diesel_rev_bio_dict[r])['categories'] for r in range(lci_nrow)]

In [142]:
allBioNames_onsite_coal = [bw.get_activity(farmxLCA_onsite_coal_rev_bio_dict[r])['name'] for r in range(lci_nrow)]
allBioCompartment_onsite_coal = [bw.get_activity(farmxLCA_onsite_coal_rev_bio_dict[r])['categories'] for r in range(lci_nrow)]

In [143]:
allBioNames_onsite_animalemi = [bw.get_activity(farmxLCA_onsite_animalemi_rev_bio_dict[r])['name'] for r in range(lci_nrow)]
allBioCompartment_onsite_animalemi = [bw.get_activity(farmxLCA_onsite_animalemi_rev_bio_dict[r])['categories'] for r in range(lci_nrow)]

In [144]:
#len(LCI_cradle_to_gate_upstream.tolist()) == len(LCI_cradle_to_gate_onsite.tolist())

In [145]:
globals()['upstream_finalLCI_farm%s' % x] = pd.DataFrame({'name' : allBioNames_upstream,
                         'categories' : allBioCompartment_upstream,
                          'value' : LCI_cradle_to_gate_upstream.tolist()}, 
                         columns=['name','categories','value'])

In [146]:
globals()['onsite_finalLCI_farm%s' % x] = pd.DataFrame({'name' : allBioNames_onsite,
                         'categories' : allBioCompartment_onsite,
                          'value' : LCI_cradle_to_gate_onsite.tolist()}, 
                         columns=['name','categories','value'])

In [147]:
globals()['onsite_diesel_finalLCI_farm%s' % x] = pd.DataFrame({'name' : allBioNames_onsite_diesel,
                         'categories' : allBioCompartment_onsite_diesel,
                          'value' : LCI_cradle_to_gate_onsite_diesel.tolist()}, 
                         columns=['name','categories','value'])

In [148]:
globals()['onsite_coal_finalLCI_farm%s' % x] = pd.DataFrame({'name' : allBioNames_onsite_coal,
                         'categories' : allBioCompartment_onsite_coal,
                          'value' : LCI_cradle_to_gate_onsite_coal.tolist()}, 
                         columns=['name','categories','value'])

In [149]:
globals()['onsite_animalemi_finalLCI_farm%s' % x] = pd.DataFrame({'name' : allBioNames_onsite_animalemi,
                         'categories' : allBioCompartment_onsite_animalemi,
                          'value' : LCI_cradle_to_gate_onsite_animalemi.tolist()}, 
                         columns=['name','categories','value'])

In [150]:
globals()['upstream_finalLCI_farm%s' % x].to_csv("farm" + farmno + "_upstreamprocess.csv")

In [151]:
globals()['onsite_finalLCI_farm%s' % x].to_csv("farm" + farmno + "_onsiteResource.csv")

In [152]:
globals()['onsite_diesel_finalLCI_farm%s' % x].to_csv("farm" + farmno + "_onsiteDiesel.csv")

In [153]:
globals()['onsite_coal_finalLCI_farm%s' % x].to_csv("farm" + farmno + "_onsiteCoal.csv")

In [154]:
globals()['onsite_animalemi_finalLCI_farm%s' % x].to_csv("farm" + farmno + "_onsiteAnimalemi.csv")